# Projeto 1 - Ciência dos Dados

Nome: Larissa Jordana

Nome: Leticia Machado

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [422]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 

In [423]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\letic\OneDrive\Área de Trabalho\SEGUNDO SEMESTRE\CIENCIA DOS DADOS\Projeto 1 - CDados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [424]:
filename = 'milka.xlsx'

In [425]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,CLASSIFICAÇÃO (Relevante (R) ou Irrelevante (I))
0,milka é o melhor chocolate do mundo 🤝,R
1,@lauraalves_0 @thmf15 esse milka é bom demaisssss,R
2,@myrelle_milka péssimo de vdd,I
3,comer minha barra de milka oreo rezando pqp,I
4,estou extremamente viciada em milka de avelã e...,R


In [426]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,CLASSIFICAÇÃO (Relevante (R) ou Irrelevante (I))
0,comendo ovinho milka de colher sabor oreo,I
1,esse milka de caramelo com avelã tá surreal,R
2,ganhei duas barras grandes da milka 😍😍,R
3,"eu so queria a porra da barra da milka, minha ...",R
4,estou no moquifo do inferno só pode🤦🏽‍♂️,I


___
## Classificador automático de sentimento


O produto escolhido foi o chocolate "milka" e foram classificados como relevantes os tweets que faziam críticas ao produto ou tinham uma opinião positiva

___
## Limpeza / Transformação das mensagens

In [427]:
#Função para limpar !-.:?;"”“*)(/|[\]
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;"”“*)(/|[\]]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return limpa(text_subbed)

In [428]:
#Função para limpar @ e http
def limpa(text):
    lista = ""
    for i in text.split():
        if i[:4]!= 'http' and i[0]!='@':
            lista+= i + " "
    return lista.strip()
                

___
## Tabela de frequência 

In [429]:
#Separando as mensagens Relevantes e Irrelevantes da nossa base de treinamento
texto_treinamentoI = train.loc[train["CLASSIFICAÇÃO (Relevante (R) ou Irrelevante (I))"]=="I", "Treinamento"]
texto_treinamentoR = train.loc[train["CLASSIFICAÇÃO (Relevante (R) ou Irrelevante (I))"]=="R","Treinamento"]

In [430]:
#Implementando a limpeza
lista_total= []
for i in train.Treinamento:
    lista_total.append(cleanup(i))
lista_total= " ".join(lista_total).split()

lista_r= []
for i in texto_treinamentoR:
    lista_r.append(cleanup(i))
lista_r= " ".join(lista_r).split()

lista_i= []
for i in texto_treinamentoI:
    lista_i.append(cleanup(i))
lista_i= " ".join(lista_i).split()

In [431]:
# Guardando as palavras como um pd.Series
serie_treinamento_R = pd.Series(lista_r)
serie_treinamento_T = pd.Series(lista_total)
serie_treinamento_I = pd.Series(lista_i)

In [432]:
len(serie_treinamento_T)

3753

#### Frequências absolutas

In [433]:
#Frequencia absoluta
tabela_treinamento_R_abs = serie_treinamento_R.value_counts()
tabela_treinamento_T_abs = serie_treinamento_T.value_counts()
tabela_treinamento_I_abs = serie_treinamento_I.value_counts()

#### Frequências relativas

In [434]:
tabela_treinamento_R_relativa = serie_treinamento_R.value_counts(True)
tabela_treinamento_T_relativa = serie_treinamento_T.value_counts(True)
tabela_treinamento_I_relativa = serie_treinamento_I.value_counts(True)

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

#### Naive-Bayes
#### $\quad P(c/x) = \frac {P(x/c) \cdot P(c)}{P(x)}$

In [435]:
#Calculo de P(R) e P(I)
probR = sum(tabela_treinamento_R_abs) / sum(tabela_treinamento_T_abs)

probI = sum(tabela_treinamento_I_abs) / sum(tabela_treinamento_T_abs)

In [436]:
#Fazendo a limpeza
lista_teste= []
for i in test.Teste:
    lista_teste.append(cleanup(i).split())

In [437]:
#Criando uma coluna nova para o classifcador
test['Classificação 2'] = ""

In [438]:
test

,Teste,CLASSIFICAÇÃO (Relevante (R) ou Irrelevante (I)),Classificação 2
0,comendo ovinho milka de colher sabor oreo,I,
1,esse milka de caramelo com avelã tá surreal,R,
2,ganhei duas barras grandes da milka 😍😍,R,
3,"eu so queria a porra da barra da milka, minha ...",R,
4,estou no moquifo do inferno só pode🤦🏽‍♂️,I,
...,...,...,...
295,@isabelcosta0 uma vez eu tava com a dieta da g...,I,
296,"comprei duas barras de milka e slc, bom demais",R,
297,@yooniemistic não milka,I,
298,@myrelle_milka parabénsss ❤️❤️❤️❤️❤️,I,


In [439]:
i= 0
#Calcula a probabilidade de uma frase ser relevante ou irrelevante por naive bayes com a suavização de laplace
for f in lista_teste:
    prob_dado_R = 1
    for c in f:
        if c in tabela_treinamento_R_abs: 
            a = (tabela_treinamento_R_abs[c] + 1)/(len(serie_treinamento_R) + len(tabela_treinamento_T_relativa))
            prob_dado_R = prob_dado_R * a
        else:
            a = 1/(len(serie_treinamento_R) + len(tabela_treinamento_T_relativa))
            prob_dado_R = prob_dado_R * a
    
    prob_dado_I = 1
    for b in f:
        if b in tabela_treinamento_I_abs:
            d = (tabela_treinamento_I_abs[b] + 1) / (len(serie_treinamento_I) + len(tabela_treinamento_T_relativa))
            prob_dado_I = prob_dado_I * d
        else:
            d = 1 / (len(serie_treinamento_I) + len(tabela_treinamento_T_relativa))
            prob_dado_I = prob_dado_I * d


    prob_R = prob_dado_R * probR
    prob_I = prob_dado_I * probI

    if prob_R > prob_I:
        test.iloc[i, 2] = 'Relevante'

    elif prob_I > prob_R:
        test.iloc[i, 2] = 'Irrelevante'
    i = i + 1

In [440]:
test.head(60)

,Teste,CLASSIFICAÇÃO (Relevante (R) ou Irrelevante (I)),Classificação 2
0,comendo ovinho milka de colher sabor oreo,I,Irrelevante
1,esse milka de caramelo com avelã tá surreal,R,Relevante
2,ganhei duas barras grandes da milka 😍😍,R,Relevante
3,"eu so queria a porra da barra da milka, minha ...",R,Irrelevante
4,estou no moquifo do inferno só pode🤦🏽‍♂️,I,Irrelevante
5,meninas a dica de hoje é harmonizar sua proibi...,R,Irrelevante
6,meus sobrinhos ganharam ovo de páscoa milka 🥺🥺🥺🥺,I,Irrelevante
7,preciso comentar aqui que aquele doce rafaelo ...,R,Relevante
8,@shakemilka milka você me odeia né ???,I,Irrelevante
9,"decidi me testar, comprei aquele chocolate mil...",R,Irrelevante


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [441]:
test

,Teste,CLASSIFICAÇÃO (Relevante (R) ou Irrelevante (I)),Classificação 2
0,comendo ovinho milka de colher sabor oreo,I,Irrelevante
1,esse milka de caramelo com avelã tá surreal,R,Relevante
2,ganhei duas barras grandes da milka 😍😍,R,Relevante
3,"eu so queria a porra da barra da milka, minha ...",R,Irrelevante
4,estou no moquifo do inferno só pode🤦🏽‍♂️,I,Irrelevante
...,...,...,...
295,@isabelcosta0 uma vez eu tava com a dieta da g...,I,Irrelevante
296,"comprei duas barras de milka e slc, bom demais",R,Relevante
297,@yooniemistic não milka,I,Irrelevante
298,@myrelle_milka parabénsss ❤️❤️❤️❤️❤️,I,Relevante


In [442]:
test["Performance"]= " "
for i in range(len(test)):
    if test.iloc[i,1]== "R" and test.iloc[i,2]== "Relevante":
        test.iloc[i,3]= "Verdadeiro Positivo"
    elif test.iloc[i,1]== "I" and test.iloc[i,2]== "Irrelevante":
        test.iloc[i,3]= "Verdadeiro Negativo"
    elif test.iloc[i,1]== "R" and test.iloc[i,2]== "Irrelevante":
        test.iloc[i,3]= "Falso Negativo"
    elif test.iloc[i,1]== "I" and test.iloc[i,2]== "Relevante":
        test.iloc[i,3]= "Falso Positivo"

In [447]:
test

,Teste,CLASSIFICAÇÃO (Relevante (R) ou Irrelevante (I)),Classificação 2,Performance
0,comendo ovinho milka de colher sabor oreo,I,Irrelevante,Verdadeiro Negativo
1,esse milka de caramelo com avelã tá surreal,R,Relevante,Verdadeiro Positivo
2,ganhei duas barras grandes da milka 😍😍,R,Relevante,Verdadeiro Positivo
3,"eu so queria a porra da barra da milka, minha ...",R,Irrelevante,Falso Negativo
4,estou no moquifo do inferno só pode🤦🏽‍♂️,I,Irrelevante,Verdadeiro Negativo
...,...,...,...,...
295,@isabelcosta0 uma vez eu tava com a dieta da g...,I,Irrelevante,Verdadeiro Negativo
296,"comprei duas barras de milka e slc, bom demais",R,Relevante,Verdadeiro Positivo
297,@yooniemistic não milka,I,Irrelevante,Verdadeiro Negativo
298,@myrelle_milka parabénsss ❤️❤️❤️❤️❤️,I,Relevante,Falso Positivo


In [443]:
p= test.Performance.value_counts(normalize= True).round(4)*100
vp =p["Verdadeiro Positivo"]
vn =p["Verdadeiro Negativo"]
fn =p["Falso Negativo"].round(4)
fp =p["Falso Positivo"].round(4)
print("{0}% Verdadeiro Positivo".format(vp))
print("{0}% Verdadeiro Negativo".format(vn))
print("{0}% Falso Negativo".format(fn))
print("{0}% Falso Positivo".format(fp))

27.33% Verdadeiro Positivo
32.0% Verdadeiro Negativo
23.33% Falso Negativo
17.33% Falso Positivo


In [444]:
#Para saber quantos tweets relevantes e quantos irrelevantes
test['CLASSIFICAÇÃO (Relevante (R) ou Irrelevante (I))'].value_counts()

R    152
I    148
Name: CLASSIFICAÇÃO (Relevante (R) ou Irrelevante (I)), dtype: int64

___
### Concluindo

O nosso projeto utilizou uma base de dados de 600 tweets, sendo eles 300 para base de treinamento e 300 para a base de teste.
A base de teste tinha 152 tweets relevantes 

In [445]:
#Distrubuição de acertos
da= pd.crosstab(test["CLASSIFICAÇÃO (Relevante (R) ou Irrelevante (I))"],test['Classificação 2'], normalize=True).round(4)*100
da

Classificação 2,Irrelevante,Relevante
CLASSIFICAÇÃO (Relevante (R) ou Irrelevante (I)),,
I,32.00,17.33
R,23.33,27.33


In [446]:
acerto1 = p["Verdadeiro Positivo"] + p["Verdadeiro Negativo"]
acerto = acerto1.round(5)
erro1 = p["Falso Positivo"] + p["Falso Negativo"]
erro = erro1.round(5)
print("A porcentagem de acerto é de {0}%".format(acerto))
print("A porcentagem de erro é de {0}%".format(erro))

A porcentagem de acerto é de 59.33%
A porcentagem de erro é de 40.66%


A performance do nosso classificador não pode ser considerada ruim, já que classificou 59,33% com precisão. Levando em conta a pequena base de dados e que não fizemos a classificação avançada.

### Mensagens com dupla negação e sarcasmo.
As mensagens que contem esses tipos de ideia são classificadas erroniamente pois são vistas de formar literal. O programa não consegue diferenciar de uma negação.

### Plano de expansão
###### Ideia 1
Aumentar a base de dados e fazer uma distribuição melhor para ter 50% dos tweets relevantes e 50% dos tweets irrelevantes pois como utilizamos a suavização de Laplace e na conta das probalidades utiliza o número de palavras que classificamos como Relevantes e Irrelevantes, os dois devem ser parecidos para não alterar o resultado da conta.
###### Devem continuar investindo
Pois a execução do nosso trabalho, visa listar feedbacks que ajuda a empresa de formar ampla. Como quais chocolates, as pessoas gostam mais e tem mais probabilidade de vendas, também tendo feedbacks construtivos, que seria formas, gostos e preços dos seus produtos que não atende todo tipo de públicos. Sendo assim, a empresa tem uma visão melhor das suas vendas e podem desenvolver melhores planejamentos e táticas de vendas, aumentando mais o lucro de vendas dos produtos e deixando seus consumidores mais contentes.

___
### Outros cenários para Naïve Bayes
1. Probabilidade de um resultado de um exame estar correta, para isso seria utilizada uma base de dados, em que teriamos os resultados das pessoas que comprovadamente tem a doença. E os resultados dos exames das pessoas que comprovadamente não tem a doença.
2. Previsão de tempo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**